In [1]:
import cv2
import numpy as np
import face_recognition
from datetime import datetime
import os
from tkinter import messagebox


In [2]:
path = "images"
img = []
Names = []
myList = os.listdir(path)
for cls in myList:
    image = cv2.imread(f'{path}/{cls}')
    img.append(image)
    Names.append(os.path.splitext(cls)[0])


def find_encodings(img):
    encode_List = []
    for im in img:
        im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
        encode_img = face_recognition.face_encodings(im)[0]
        encode_List.append(encode_img)
    return encode_List


encoding_known_faces = find_encodings(img)
print('Encoding is done')


Encoding is done


In [3]:
def attendance(person_name):
    with open('Attendance.csv', 'r+') as f:
        data_list = f.readlines()
        Name_list = []
        for line in data_list:
            find_entry = line.split(',')
            Name_list.append(find_entry[0])
        if person_name not in Name_list:
            present = datetime.now()
            date = present.strftime('%H:%M:%S')
            f.writelines(f'\n{person_name},{date}')


In [4]:
#Finding face matches using webcam
capture = cv2.VideoCapture(0)

record = True

while record:
    success, im = capture.read()
    im_resize = cv2.resize(im,(0,0),None,0.25,0.25)
    im_resize=cv2.cvtColor(im_resize,cv2.COLOR_BGR2RGB)
    
    
    #finding the face locations in current frame
    face = face_recognition.face_locations(im_resize)
    encode_current_face = face_recognition.face_encodings(im_resize,face)
     
     #grabbing one face location from current face list and grabbing encode_face from encode_current_face
    for encode_face,face_loc in zip(encode_current_face,face):
        #matching/comparing faces
        face_match = face_recognition.compare_faces(encoding_known_faces,encode_face)
        #finding face distance 
        face_distance = face_recognition.face_distance(encoding_known_faces,encode_face)
        #print(face_distance)
        # lowest distance image is best match
        face_match_index = np.argmin(face_distance)

        #Displaying rectangle bos around face and writing the person name
        if face_match[face_match_index]:
            person_name = Names[face_match_index].upper()
            #print(person_name)
            y1,x2,y2,x1 = face_loc
            y1,x2,y2,x1 = y1*4,x2*4,y2*4,x1*4
            cv2.rectangle(im,(x1,y1),(x2,y2),(0,255,0),2)
            cv2.rectangle(im,(x1,y2-35),(x2,y2),(0,255,0),cv2.FILLED)
            cv2.putText(im,person_name,(x1+6,y2-6),cv2.FONT_HERSHEY_TRIPLEX,1,(255,0,0),2)
            
            attendance(person_name)

            record = False
            capture.release()
            cv2.destroyAllWindows()

    if record:
        cv2.imshow('web_Cam',im)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            record = False
            capture.release()
            cv2.destroyAllWindows()
    else:
        break

messagebox.showinfo("Attendance", "Attendance Successful")

'ok'